In [1]:
import numpy as np
from h5py import File
import scipy.io #Used to load the OCTAVE *.mat files
import numpy as np
import sys
sys.path.append ('../src')
from NeuralNetwork_v2 import SigmoidF,ReluF,TanhF,LReluF,Activation_V2,Linear,Layer,NNClassifier_V2,SoftmaxF, BinaryCrossEntropy, CategoricalCrossEntropy
from ML_utils import softmax,sigmoid,UTIL_formatY,backward_prop,backpropagation
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix,classification_report

In [2]:
MNIST_data = File("data/MNISTdata.hdf5", 'r')
x_train = np.float32(MNIST_data['x_train'][:])
y_train = np.int32(np.array(MNIST_data['y_train'][:, 0])).reshape(-1, 1)
x_test  = np.float32(MNIST_data['x_test'][:])
y_test  = np.int32(np.array(MNIST_data['y_test'][:, 0])).reshape(-1, 1)
MNIST_data.close()

# stack together for next step
X = np.vstack((x_train, x_test))
y = np.vstack((y_train, y_test))

# one-hot encoding
digits = 10
examples = y.shape[0]
y = y.reshape(1, examples)
Y_new = np.eye(digits)[y.astype('int32')]
Y_new = Y_new.T.reshape(digits, examples)


# number of training set
m = 60000
m_test = X.shape[0] - m
X_train, X_test = X[:m].T, X[m:].T
Y_train, Y_test = Y_new[:, :m], Y_new[:, m:]


# shuffle training set
shuffle_index = np.random.permutation(m)
X_train, Y_train = X_train[:, shuffle_index], Y_train[:, shuffle_index]

## NNClassifier Test

In [4]:
from NeuralNetwork import NNClassifier
from ML_utils import softmax,sigmoid,UTIL_formatY,backward_prop,backpropagation
nn_config={'n_a1':784,'n_a2':128 ,'n_a3':64,'n_a4':25 , 'n_a5':10}   # Configuración de red NN , input layer , hidder layers , output layer
sgd_dict =   {'steps':25,'learning_rate':0.5,'mini_batch_size':2**8}
opt_dict =   {'maxiter':500,'algorithm' : 'TNC'}
activ = {'activation_a2':sigmoid,'activation_a3':sigmoid,'activation_a4':sigmoid,'activation_a5':softmax}
method = 'miniBatchGD'
if method == 'Optimize':
    midict = opt_dict
else:
    midict = sgd_dict
    
nn = NNClassifier(optimization=method,bias=True,nn_config=nn_config,activ=activ,debug=False,kargs=midict)

costs = nn.optimize (X_train.T,Y_train.T,l2_lambda=0.0)
prediction,_ = nn.forward_prop (X_train.T)
result = np.argmax(prediction,axis=1).reshape(-1,1)
y = np.argmax(Y_train.T,axis=1).reshape(-1,1)

accuracy = np.mean(y==result) * 100
print ('Trainig set accuracy :' , accuracy  )

test_predicted,_ = nn.forward_prop (X_test.T)
result = np.argmax(test_predicted,axis=1).reshape(-1,1)
y = np.argmax(Y_test.T,axis=1).reshape(-1,1)
accuracy = np.mean(y==result) * 100
print ('Test set accuracy :' , accuracy  )

print(f"Classification report for classifier {nn}:\n"
      f"{classification_report(y,result)}\n")



LR = 0.5: 100%|████████████████████████████████████████████████████████████████████████| 25/25 [03:12<00:00,  7.69s/it]


Trainig set accuracy : 98.01166666666667
Test set accuracy : 96.76
Classification report for classifier <NeuralNetwork.NNClassifier object at 0x000001C0274AA490>:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.98      0.99      0.98      1135
           2       0.97      0.97      0.97      1032
           3       0.97      0.95      0.96      1010
           4       0.98      0.96      0.97       982
           5       0.97      0.96      0.96       892
           6       0.97      0.98      0.97       958
           7       0.97      0.96      0.96      1028
           8       0.95      0.97      0.96       974
           9       0.95      0.97      0.96      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000




## NNClassifier_V2

In [3]:
layer1 = Layer (128,784,activationF=SigmoidF,keep_prob=1.0)
layer2 = Layer (64,128,activationF=SigmoidF,keep_prob=1.0)
layer3 = Layer (25,64,activationF=SigmoidF,keep_prob=1.0)
layer4 = Layer (10,25,activationF=SoftmaxF,keep_prob=1.0)
layers = [layer1,layer2,layer3,layer4]
nn = NNClassifier_V2 (layers,costFunction=CategoricalCrossEntropy,kargs={'steps':25,'learning_rate':0.5,'mini_batch_size':2**8},print_cost=False)
costs=nn.optimize (X_train,Y_train,l2_lambda = 0.0,method='miniBatchGD')
train_prediction = nn.forward (X_train)
result = np.argmax(train_prediction.T,axis=1).reshape(-1,1)
y = np.argmax(Y_train.T,axis=1).reshape(-1,1)
accuracy = np.mean(y==result) * 100
print ('Trainig set accuracy :' , accuracy  )


test_predicted = nn.forward (X_test)
result = np.argmax(test_predicted.T,axis=1).reshape(-1,1)
y = np.argmax(Y_test.T,axis=1).reshape(-1,1)
accuracy = np.mean(y==result) * 100
print ('Test set accuracy :' , accuracy  )

print(f"Classification report for classifier {nn}:\n"
      f"{classification_report(y,result)}\n")

LR = 0.5: 100%|████████████████████████████████████████████████████████████████████████| 25/25 [01:58<00:00,  4.74s/it]


Trainig set accuracy : 98.21666666666667
Test set accuracy : 96.97
Classification report for classifier <NeuralNetwork_v2.NNClassifier_V2 object at 0x000001D6786705B0>:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       980
           1       0.98      0.99      0.98      1135
           2       0.97      0.98      0.97      1032
           3       0.96      0.97      0.96      1010
           4       0.97      0.96      0.97       982
           5       0.97      0.96      0.97       892
           6       0.98      0.98      0.98       958
           7       0.97      0.96      0.96      1028
           8       0.96      0.97      0.97       974
           9       0.96      0.95      0.95      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000


